In [1]:
import sys
import yaml
from sklearn.model_selection import train_test_split

sys.path.append('../../../../')

In [2]:
from scripts.utils import load_data, preprocess_data_label_encoder, create_sequences, train_and_test, simulate_predictions, simulate_predictions_from_pretrained

No GPU available. Running on CPU.


In [3]:
from src.models.cnn import build_model

In [4]:
YAML_FILE = '../../../../configs/games/tic-tac-toe/cnn/1k.yml'

In [5]:
with open(YAML_FILE, 'r') as f:
    config = yaml.safe_load(f)
print(config)

{'name': '/games/tic-tac-toe/cnn/cnn_1k', 'model': {'type': 'CNN', 'num_filters': 128, 'filter_sizes': [3, 4, 5]}, 'training': {'batch_size': 64, 'learning_rate': 0.001, 'num_epochs': 5}, 'data': {'game': 'tic-tac-toe', 'sequence_length': 20, 'max_event_length': 50, 'path': '/games/tic-tac-toe/1k_single_agent.csv'}}


In [6]:
model_params = {
    'num_filters': config['model']['num_filters'],
    'filter_sizes': config['model']['filter_sizes']
}

In [7]:
# Load and preprocess data
df = load_data(f'../../../../data/processed/{config['data']['path']}')
df, e_event, e_agent, e_context = preprocess_data_label_encoder(df)

# Create sequences
sequence_length = config['data']['sequence_length']
X, y = create_sequences(df, sequence_length)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
model = train_and_test(
    X_train, 
    X_test, 
    y_train, 
    y_test, 
    e_event, 
    e_agent, 
    e_context, 
    name=config['name'], 
    epochs=config['training']['num_epochs'], 
    batch_size=config['training']['batch_size'],
    model_params=model_params,
    build_model=build_model,
)

Epoch 1/5
120/120 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - loss: 4.4204 - output_1_accuracy: 0.7683 - output_2_accuracy: 0.4056 - output_3_accuracy: 0.0973 - val_loss: 4.1926 - val_output_1_accuracy: 0.8010 - val_output_2_accuracy: 0.4182 - val_output_3_accuracy: 0.0875
Epoch 2/5
120/120 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 4.2255 - output_1_accuracy: 0.7875 - output_2_accuracy: 0.4155 - output_3_accuracy: 0.0960 - val_loss: 4.2003 - val_output_1_accuracy: 0.8010 - val_output_2_accuracy: 0.4432 - val_output_3_accuracy: 0.0969
Epoch 3/5
120/120 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 4.1838 - output_1_accuracy: 0.7905 - output_2_accuracy: 0.4301 - output_3_accuracy: 0.1069 - val_loss: 4.1345 - val_output_1_accuracy: 0.8010 - val_output_2_accuracy: 0.4495 - val_output_3_accuracy: 0.0922
Epoch 4/5
120/120 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 4.1225 - output_1_accuracy: 0.7833 - output_2_accuracy: 0.4400 - output_3_accuracy: 0.1338 - val_loss: 4.1091 - val_output_1_accuracy: 0.8010 - v

Test Results:
loss: 4.0009
compile_metrics: 0.8068


In [9]:
simulate_predictions(
    data=X,
    model=model,
    e_event=e_event,
    e_agent=e_agent,
    e_context=e_context,
    n = 15,
    k = config['data']['sequence_length']
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Initial sequence:
MOVE, O, 2,0
GAME_END, system, O
GAME_START, system, New Game
MOVE, X, 1,1
MOVE, O, 1,0
MOVE, X, 2,2
MOVE, O, 0,0
MOVE, X, 2,1
MOVE, O, 1,2
MOVE, X, 0,1
GAME_END, system, X
GAME_START, system, New Game
MOVE, X, 1,1
MOVE, O, 2,1
MOVE, X, 2,0
MOVE, O, 0,2
MOVE, X, 2,2
MOVE, O, 0,0
MOVE, X, 1,2
MOVE, O, 0,1

Predicted moves:
Move 1: MOVE, X, 0,0
Move 2: MOVE, system, New Game
Move 3: MOVE, system, New Game
Move

In [12]:
simulate_predictions_from_pretrained(
    data=X,
    modelpath=f'../../../../models/{config['name']}.h5',
    e_event=e_event,
    e_agent=e_agent,
    e_context=e_context,
    n = 15,
    k = config['data']['sequence_length']
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Initial sequence:
GAME_START, system, New Game
MOVE, X, 0,1
MOVE, O, 2,2
MOVE, X, 2,0
MOVE, O, 1,1
MOVE, X, 1,2
MOVE, O, 2,1
MOVE, X, 0,2
MOVE, O, 1,0
MOVE, X, 0,0
GAME_END, system, X
GAME_START, system, New Game
MOVE, X, 1,2
MOVE, O, 2,2
MOVE, X, 1,1
MOVE, O, 0,0
MOVE, X, 1,0
GAME_END, system, X
GAME_START, system, New Game
MOVE, X, 0,0

Predicted moves:
Move 1: MOVE, O, 0,0
Move 2: MOVE, X, 2,0
Move 3: MOVE, O, 2,0
Move 4: 